### Note:

Add code to save graphs in showPlot function



In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re  
import random
import numpy as np
import os
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import collections
from itertools import dropwhile

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
PATH = "./"


In [3]:
#!pip3 install sacrebleu
from sacrebleu import corpus_bleu


In [4]:
#read in chinese-english pairs
#read in chinese-english pairs
lines_zh = open(PATH+'iwslt-zh-en/train.tok.zh',encoding = 'utf-8').read().strip().split('\n')
lines_en = open(PATH+'iwslt-zh-en/train.tok.en',encoding = 'utf-8').read().strip().split('\n')
lines_zh_test = open(PATH+'iwslt-zh-en/test.tok.zh',encoding = 'utf-8').read().strip().split('\n')
lines_en_test = open(PATH+'iwslt-zh-en/test.tok.en',encoding = 'utf-8').read().strip().split('\n')
lines_zh_val = open(PATH+'iwslt-zh-en/dev.tok.zh',encoding = 'utf-8').read().strip().split('\n')
lines_en_val = open(PATH+'iwslt-zh-en/dev.tok.en',encoding = 'utf-8').read().strip().split('\n')

In [5]:
def delect_least_common_words(list_sent, threshold = 5):
    ret_list =[]
    for x in list_sent:
        ret_list += x.split()
    ret_dic = collections.Counter(ret_list)
    
    #print (ret_dic["&amp;"])
    #print (ret_dic["&apos;"])
    #print (ret_dic["&quot;"])
    #print (ret_dic["&#91"])
    for key, count in dropwhile(lambda key_count: key_count[1] >= threshold, ret_dic.most_common()):
        
        del ret_dic[key]
        
        
    return list(ret_dic.keys())

In [6]:
zh_words = delect_least_common_words(lines_zh)
en_words = delect_least_common_words(lines_en)

In [7]:
len(zh_words)

34443

In [8]:
words_to_load = 100000
PAD_IDX = 0
UNK_IDX = 1
SOS_IDX = 2
EOS_IDX = 3

with open('../cc.zh.300.vec') as f:
    loaded_embeddings_ft = np.zeros((words_to_load+3, 300))
    words_ft = {}
    idx2words_ft = {}
    ordered_words_ft = []
    ordered_words_ft.extend(['<pad>', '<unk>', '<s>'])
    loaded_embeddings_ft[0,:] = np.zeros(300)
    loaded_embeddings_ft[1,:] = np.random.normal(size = 300)
    loaded_embeddings_ft[2,:] = np.random.normal(size = 300)
    for i, line in enumerate(f):
        if i >= words_to_load: 
            break
        s = line.split()
        loaded_embeddings_ft[i+3, :] = np.asarray(s[1:])
        words_ft[s[0]] = i+3
        idx2words_ft[i+3] = s[0]
        ordered_words_ft.append(s[0])
    length = len(np.setdiff1d(zh_words, ordered_words_ft))
    tmp_embeddings = np.zeros((length, 300))
    for idx, word in enumerate(np.setdiff1d(zh_words, ordered_words_ft)):
        words_ft[word] = idx+words_to_load+3
        idx2words_ft[idx+words_to_load+3] = word
        tmp_embeddings[idx, :] = np.random.normal(size = 300)
    loaded_embeddings_ft = np.concatenate((loaded_embeddings_ft, tmp_embeddings), axis = 0)
    words_ft['<pad>'] = PAD_IDX
    words_ft['<unk>'] = UNK_IDX
    words_ft['<s>'] = SOS_IDX
    idx2words_ft[PAD_IDX] = '<pad>'
    idx2words_ft[UNK_IDX] = '<unk>'
    idx2words_ft[SOS_IDX] = '<s>'
    
ordered_words_ft = list(words_ft.keys())

In [9]:
#English embedding
with open(PATH+'wiki-news-300d-1M.vec') as f:
    loaded_embeddings_ft_en = np.zeros((words_to_load+4, 300))
    words_ft_en = {}
    idx2words_ft_en = {}
    ordered_words_ft_en = []
    ordered_words_ft_en.extend(['<pad>', '<unk>', '<s>', '</s>'])
    loaded_embeddings_ft_en[0,:] = np.zeros(300)
    loaded_embeddings_ft_en[1,:] = np.random.normal(size = 300)
    loaded_embeddings_ft_en[2,:] = np.random.normal(size = 300)
    loaded_embeddings_ft_en[3,:] = np.random.normal(size = 300)
    for i, line in enumerate(f):
        if i >= words_to_load: 
            break
        s = line.split()
        loaded_embeddings_ft_en[i+4, :] = np.asarray(s[1:])
        words_ft_en[s[0]] = i+4
        idx2words_ft_en[i+4] = s[0]
        ordered_words_ft_en.append(s[0])
    length = len(np.setdiff1d(en_words, ordered_words_ft_en))
    tmp_embeddings = np.zeros((length, 300))
    for idx, word in enumerate(np.setdiff1d(en_words, ordered_words_ft_en)):
        words_ft_en[word] = idx+words_to_load+4
        idx2words_ft_en[idx+words_to_load+4] = word
        tmp_embeddings[idx, :] = np.random.normal(size = 300)
    loaded_embeddings_ft_en = np.concatenate((loaded_embeddings_ft_en, tmp_embeddings), axis = 0)
    words_ft_en['<pad>'] = PAD_IDX
    words_ft_en['<unk>'] = UNK_IDX
    words_ft_en['<s>'] = SOS_IDX
    words_ft_en['</s>'] = EOS_IDX
    idx2words_ft_en[PAD_IDX] = '<pad>'
    idx2words_ft_en[UNK_IDX] = '<unk>'
    idx2words_ft_en[SOS_IDX] = '<s>'
    idx2words_ft_en[EOS_IDX] = '</s>'
    
ordered_words_ft_en = list(words_ft_en.keys())

In [10]:
assert len(idx2words_ft) == len(words_ft)
assert len(loaded_embeddings_ft) == len(words_ft)
assert len(idx2words_ft_en) == len(words_ft_en)
assert len(loaded_embeddings_ft_en) == len(words_ft_en)
assert len(ordered_words_ft_en) == len(loaded_embeddings_ft_en)

In [11]:
#add sos and eos in each sentence
def add_sos_eos(lines):
    
    train = []
    for l in lines:
        l = '<s> ' + l + ' </s>'
        train.append(l)
    return train
zh_train = add_sos_eos(lines_zh)    
en_train = add_sos_eos(lines_en)
zh_test = add_sos_eos(lines_zh_test)
en_test = add_sos_eos(lines_en_test)
zh_val = add_sos_eos(lines_zh_val)
en_val = add_sos_eos(lines_en_val)

In [12]:
en_train[6]

'<s> And the problem , I think , is that we take the ocean for granted . </s>'

In [13]:
# convert token to id in the dataset
def token2index_dataset(tokens_data,eng = False):
    indices_data = []
    for tokens in tokens_data:
        index_list = []
        for token in tokens.split():
            if eng == False:
                try:
                    index_list.append(words_ft[token])
                except KeyError:
                    index_list.append(UNK_IDX)
            else:
                try:
                    index_list.append(words_ft_en[token])
                except KeyError:
                    index_list.append(UNK_IDX)
        indices_data.append(index_list)
    return indices_data

In [14]:
zh_train_indices = token2index_dataset(zh_train)
en_train_indices = token2index_dataset(en_train,eng = True)
zh_val_indices = token2index_dataset(zh_val)
en_val_indices = token2index_dataset(en_val,eng = True)

In [15]:
#max_sentence_length
length_of_en = [len(x.split()) for x in en_train]
max_sentence_length_en = sorted(length_of_en)[-int(len(length_of_en)*0.01)]
length_of_zh = [len(x.split()) for x in zh_train]
max_sentence_length_zh = sorted(length_of_zh)[-int(len(length_of_zh)*0.01)]


In [16]:
max_sentence_length_en

74

In [17]:
#Create Data Loader
import torch
from torch.utils.data import Dataset

class load_dataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    
    def __init__(self, data_list_s1,data_list_s2):
        """
        @param data_list_zh: list of Chinese tokens 
        @param data_list_en: list of English tokens as TARGETS
        """
        self.data_list_s1 = data_list_s1
        self.data_list_s2 = data_list_s2
        
        assert (len(self.data_list_s1) == len(self.data_list_s2))

    def __len__(self):
        return len(self.data_list_s1)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        
        token_idx_s1 = self.data_list_s1[key][:max_sentence_length_zh]
        token_idx_s2 = self.data_list_s2[key][:max_sentence_length_en]
        return [token_idx_s1, token_idx_s2, len(token_idx_s1), len(token_idx_s2)]

def collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    data_list_s1 = []
    data_list_s2 = []
    length_list_s1 = []
    length_list_s2 = []
    for datum in batch:
        length_list_s1.append(datum[2])
        length_list_s2.append(datum[3])
        padded_vec_zh = np.pad(np.array(datum[0]), 
                                pad_width=((0,max_sentence_length_zh-datum[2])), 
                                mode="constant", constant_values=0)
        padded_vec_en = np.pad(np.array(datum[1]), 
                                pad_width=((0,max_sentence_length_en-datum[3])), 
                                mode="constant", constant_values=0)
        data_list_s1.append(padded_vec_zh[:max_sentence_length_zh])
        data_list_s2.append(padded_vec_en[:max_sentence_length_en])
    #print(type(data_list_s1[0]))
    
    return [torch.from_numpy(np.array(data_list_s1)).to(device), torch.from_numpy(np.array(data_list_s2)).to(device),
            torch.LongTensor(length_list_s1).to(device), torch.LongTensor(length_list_s2).to(device)]
    

In [18]:
BATCH_SIZE = 100
EMBEDDING_SIZE = 300 # fixed as from the input embedding data

train_dataset = load_dataset(zh_train_indices, en_train_indices)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=collate_func,
                                           shuffle=True)

val_dataset = load_dataset(zh_val_indices, en_val_indices)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=collate_func,
                                           shuffle=False)

### Without Attention

In [19]:
class EncoderRNN(nn.Module):
    def __init__(self, emb_dim, hidden_size, embed= torch.from_numpy(loaded_embeddings_ft).float(),num_layers=1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.emb_dim = emb_dim
        self.num_layers = num_layers 

        self.embedding = nn.Embedding.from_pretrained(embed, freeze=False)
        self.gru = nn.GRU(emb_dim, hidden_size,num_layers=num_layers,batch_first=True)

    def forward(self, data, hidden):
        
        batch_size, seq_len = data.size()
        
        embed = self.embedding(data)
        
        output, hidden = self.gru(embed,hidden)
        #hidden = [n layers * n directions =1 , batch_size, hidden_size ]
        
        return output, hidden

    # initialize the hidden with random numbers
    def initHidden(self,batch_size):
        return torch.randn(self.num_layers, batch_size, self.hidden_size,device=device)

In [20]:
class DecoderRNN(nn.Module):
    def __init__(self,emb_dim,hidden_size, output_size, embed= torch.from_numpy(loaded_embeddings_ft_en).float(),num_layers=1):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers 
        self.output_size = output_size

        #self.embedding = nn.Embedding(output_size, hidden_size)
        self.embedding = nn.Embedding.from_pretrained(embed, freeze=False)
        self.gru = nn.GRU(emb_dim, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, data, hidden):
        embed = self.embedding(data)

        #print("decoder",embed.size())
        
        output = F.relu(embed)
        #print("decoder",output.size())
        
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        
        return output, hidden

    def initHidden(self,batch_size):
        return torch.randn(self.num_layers, batch_size, self.hidden_size,device=device)

In [21]:
teacher_forcing_ratio = 1
#input_tensor: list of sentence tensor
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer,
          criterion, eee):
    
    ### target_tensor [batch size, max_sentence_length_en = 377] ###
    ### target_tensor [batch size, max_sentence_length_zh = 220] ###
    batch_size_1, input_length = input_tensor.size()
    batch_size_2, target_length = target_tensor.size()
    #print ("target length ", target_length)
    
    
    encoder_hidden = encoder.initHidden(batch_size_1)
    #print ("encoder hidden init, ",h.shape)
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    loss = 0
    ### encoder_hidden: 1 * batch * hidden size ### 
    ### encoder_output: batch size * max_sentence_length_zh * hidden size ### 
    _, encoder_hidden = encoder(input_tensor, encoder_hidden)
    #print ("encoder output, ", encoder_output.shape)

    decoder_input = torch.tensor(np.array([[SOS_IDX]]*batch_size_1).reshape(1,batch_size_1),device=device)
    decoder_hidden = encoder_hidden
    #h1,c1 = encoder_hidden
    #print ("encoder hidden,", h1.shape)

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    #print(use_teacher_forcing)
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            
            ### decoder_output: [batchsize,5000] ###
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            
            #print ("decoder output, ",decoder_output.shape)
            #print ("target_tensor, ",len(target_tensor[:,di]))
            loss += criterion(decoder_output, target_tensor[:,di])
            decoder_input = target_tensor[:,di].unsqueeze(0)  # Teacher forcing
            
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
                        
            ### decoder_output [batch size, 50003]  ###
            
            ### topi is a [batch size, 1] tensor first we remove the size 1
            ### demension then we add it at the beginning using squeeze
            
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            
            ### decoder_input [1, batch size]  ###
            decoder_input = decoder_input.unsqueeze(0)
 
            loss += criterion(decoder_output, target_tensor[:,di])

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [22]:
from torch.optim.lr_scheduler import StepLR, LambdaLR
import pickle
def trainIters(encoder, decoder, n_iters, folder,lr_decrease = False,print_every=1, plot_every=100, evaluate_every = 50,read_in_model = False,learning_rate=0.001,early_stop_tol = 10e-7):
    start = time.time()
    plot_losses = []
    plot_val = []
    
    loss_history = []
   
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    patience = 0
    
    early_stopped = False
    current_best_bleu = 0
    
    best_encoder = encoder.state_dict()
    best_decoder = decoder.state_dict()
    
    
    #--------------------------------------------	
    #	
    #    LOAD MODELS	
    #	
    #--------------------------------------------	
    	
        
    
    if not os.path.exists(folder):	
        os.makedirs(folder)	

    if read_in_model == True:
        if os.path.exists(folder+'/Encoder'):	
            print('---------------------------------------------------------------------')	
            print('----------------Readind trained model---------------------------------')	
            print('---------------------------------------------------------------------')	

            #read trained models	
            encoder.load_state_dict(torch.load(folder+"/Encoder"))
            decoder.load_state_dict(torch.load(folder+"/Decoder"))	

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    
    if lr_decrease == True:
        encoder_scheduler = StepLR(encoder_optimizer, step_size=3, gamma=0.8)
        decoder_scheduler = StepLR(decoder_optimizer, step_size=3, gamma=0.8)
    
    
    criterion = nn.CrossEntropyLoss()
    #criterion_val = nn.CrossEntropyLoss()
    
    last_val = 0
    for iter in range(1, n_iters + 1):
        if lr_decrease == True:
            encoder_scheduler.step()
            decoder_scheduler.step()
        for i, (data_s1, data_s2, lengths_s1, lengths_s2) in enumerate(train_loader):
            input_tensor = data_s1
            target_tensor = data_s2
            #print("train",target_tensor.size())
            loss = train(input_tensor, target_tensor, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion,i)
            print_loss_total += loss
            plot_loss_total += loss

            if i % print_every == 0:
                if i != 0:
                    print_loss_avg = print_loss_total / print_every
                    print_loss_total = 0
                    print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                                 iter, iter / n_iters * 100, print_loss_avg))
                    loss_history.append(print_loss_avg)
                else:
                    print_loss_total = 0
                
            if i % plot_every == 0:
                if i != 0:
                    plot_loss_avg = plot_loss_total / plot_every
                    plot_losses.append(plot_loss_avg)
                    plot_loss_total = 0
                    
                    
                else:
                    plot_loss_total = 0
                
            if i % evaluate_every == 0:
                if i != 0:
                    bleu_score,output_words = evaluate(val_loader, encoder, decoder)
                    if bleu_score > current_best_bleu:
                        current_best_bleu = bleu_score
                        
                        best_encoder = encoder.state_dict()
                        best_decoder = decoder.state_dict()
                        
                    plot_val.append(bleu_score)
                    #print ("BLEU: ",bleu_score)
                    
                    if bleu_score <= current_best_bleu:
                        patience += 1
                        
                    elif bleu_score > current_best_bleu and np.abs(bleu_score - current_best_bleu)/float(current_best_bleu) < early_stop_tol:
                        patience += 1
                    
                    else:
                        patience = 0
                        
                        
                    
                    
                    """
                    #If new bleu score is lower than last time
                    if bleu_score <= last_val:
                        patience += 1
                    #or does not improve by enough percentage
                    elif bleu_score > last_val and np.abs(bleu_score - last_val)/float(last_val) < early_stop_tol:
                        
                        patience += 1
                    #bleu score increased since last time
                    else:
                        #reset patience
                        patience = 0
                            
                    """    
                    if patience == 10:
                       
                        torch.save(best_encoder,folder +"/Encoder")
                        torch.save(best_decoder,folder +"/Decoder")
                        early_stopped = True
                        patience = 0
            
                        
                    last_val = bleu_score
                 
        if early_stopped == False:
        
            # Save the model for every epoch
            print('---------------------------------------------------------------------')	
            print('----------------Saving trained model---------------------------------')	
            print('---------------------------------------------------------------------')	

            torch.save(encoder.state_dict(),folder +"/Encoder")
            torch.save(decoder.state_dict(),folder +"/Decoder")
            
    with open(folder+"/loss_hist", 'wb') as f:
         pickle.dump(loss_history, f)
    with open(folder+"/bleu_hist", 'wb') as f:
         pickle.dump(plot_val, f)
    showPlot(plot_losses,title = "Train Loss",name = folder+"/loss.jpeg")
    showPlot(plot_val, title = "BLEU Score on Validation Set",name = folder+"/bleu.jpeg")
    return plot_losses


In [23]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline

def showPlot(points,title,name):
    plt.figure()
    
    plt.plot(points)
    plt.title(title)
    plt.savefig(name)
    
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


In [24]:
#loader can be test_loader or val_loader
def evaluate(loader, encoder, decoder, after_train_mode = False,beam = False, beam_k = 1):
    bleu_score_list = []
    big_pred_list = []
    big_ref_list = []
    with torch.no_grad():
        for i, (data_s1, data_s2, lengths_s1, lengths_s2) in enumerate(loader):
            input_tensor = data_s1
            input_length = input_tensor.size()[0]
            #sentence_length to the output length
            sentence_length = data_s2.size()[1]
            encoder_hidden = encoder.initHidden(input_length)

            _, encoder_hidden = encoder(input_tensor, encoder_hidden)
            
            #decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
            decoder_input = torch.tensor(np.array([[SOS_IDX]]*input_length).reshape(1,input_length),device=device)

            decoder_hidden = encoder_hidden

            #decoder_attentions = torch.zeros(sentence_length, sentence_length)
            decoded_words_eval = []
            sequences = [[list(), 1.0]]*input_length
            for di in range(sentence_length):
                decoded_words_sub = []
                decoder_output, decoder_hidden = decoder(
                    decoder_input, decoder_hidden)
                # decoder_attentions[di] = decoder_attention.data
                # topk(1) - softmax probability maximum
                if beam == True:
                    pass
#                     topv, topi = decoder_output.data.topk(beam_k)
#                     #batch loop
#                     C = []
#                     for idx, ind in enumerate(topi):
#                         H, _ = sequences[idx]
#                         for ele in ind:
#                             if ele.item() == EOS_IDX:
#                                 H.append('<EOS>')
#                             else:
#                                 H.append(idx2words_ft_en[ele.item()])
                         
                else:
                    topv, topi = decoder_output.data.topk(1) 
                    
                #batch loop
                
                
                for ind in topi:
                    
                    if ind.item() == EOS_IDX:
                        
                        decoded_words_sub.append(idx2words_ft_en[EOS_IDX])
                        
                    else:
                        decoded_words_sub.append(idx2words_ft_en[ind.item()])
                    
                
                decoded_words_eval.append(decoded_words_sub)
                
                #swap dimensions of decoded_words to [batch_size * 377]
                
                #decoded_words_new = [[i for i in ele] for ele in list(zip(*decoded_words_eval))]

                #change the dimension
                decoder_input = topi.squeeze().detach()
                decoder_input = decoder_input.unsqueeze(0)
            
            
            pred_num = 0
            listed_predictions = []
            
            
            decoded_words_new = [[i for i in ele] for ele in list(zip(*decoded_words_eval))]
            
            for token_list in decoded_words_new:
                sent = ' '.join(str(token) for token in token_list if token!="<pad>" and token!="<s>" and token!="</s>")
                #print (sent)
                listed_predictions.append(sent)
                #print (sent)
                pred_num += 1
                
            ref_num = 0
            listed_reference = []
            for ele in data_s2:
                sent = index2token_sentence(ele)
                #print (tokens)
                #sent = ' '.join(tokens)
                #print (sent)
                listed_reference.append(sent)
                ref_num += 1
            
            big_pred_list += listed_predictions
            big_ref_list += listed_reference
            
            assert len(big_pred_list) == len(big_ref_list)
            
            
            #uncommon to print prediction and reference
            #print (listed_predictions)
            #print (listed_reference)
        bleu_score = corpus_bleu(big_pred_list,[big_ref_list]).score
        
        if after_train_mode == True:
            for idx,ele in enumerate(big_pred_list):
                print (ele)
                print (big_ref_list[idx])
                print ("\n")
                
                
    print('BLEU Score is %s' % (str(bleu_score)))
        

    return bleu_score, decoded_words_new
    
def index2token_batch(list_of_list):
    return ' '.join(idx2words_ft_en[r.item()] for v in list_of_list for r in v if r.item()!=PAD_IDX)
def index2token_sentence(sentence_batch):
    return ' '.join(idx2words_ft_en[sent.item()] for sent in sentence_batch if sent.item()!=PAD_IDX and sent.item()!=SOS_IDX and sent.item()!=EOS_IDX)

In [25]:
score_list, output_words, attentions = evaluate(val_loader, encoder1, decoder1,after_train_mode =True)

NameError: name 'encoder1' is not defined

In [ ]:
hidden_size = 200
encoder1 = EncoderRNN(EMBEDDING_SIZE,hidden_size).to(device)
decoder1 = DecoderRNN(EMBEDDING_SIZE,hidden_size, len(ordered_words_ft)).to(device)

##UNCOMMENT TO TRAIN THE MODEL
trainIters(encoder1, decoder1, 20,'hidden_200_lr_0.01', lr_decrease = True,print_every=50,plot_every = 100, evaluate_every = 250,learning_rate=0.01)

0m 41s (- 13m 11s) (1 5%) 2.3992
1m 22s (- 26m 5s) (1 5%) 1.7279
2m 3s (- 39m 0s) (1 5%) 1.6682
2m 43s (- 51m 55s) (1 5%) 1.6775
3m 24s (- 64m 50s) (1 5%) 1.6379


BLEU Score is 0.8386282924722102
4m 14s (- 80m 42s) (1 5%) 1.6074
4m 55s (- 93m 37s) (1 5%) 1.5858
5m 36s (- 106m 32s) (1 5%) 1.5758
6m 17s (- 119m 27s) (1 5%) 1.5280
6m 58s (- 132m 22s) (1 5%) 1.5175


BLEU Score is 0.9180780323941934
7m 48s (- 148m 18s) (1 5%) 1.5127
8m 29s (- 161m 13s) (1 5%) 1.5197
9m 9s (- 174m 8s) (1 5%) 1.5500
9m 50s (- 187m 3s) (1 5%) 1.4814
10m 31s (- 199m 58s) (1 5%) 1.4584


BLEU Score is 2.2424401687117235
11m 21s (- 215m 49s) (1 5%) 1.4592
12m 2s (- 228m 45s) (1 5%) 1.4479
12m 43s (- 241m 40s) (1 5%) 1.4281
13m 23s (- 254m 35s) (1 5%) 1.4177
14m 4s (- 267m 30s) (1 5%) 1.3911


BLEU Score is 2.9742970762127943
14m 54s (- 283m 22s) (1 5%) 1.3904
15m 35s (- 296m 17s) (1 5%) 1.3642
16m 16s (- 309m 12s) (1 5%) 1.3644
16m 57s (- 322m 7s) (1 5%) 1.3554
17m 38s (- 335m 2s) (1 5%) 1.3603


...

BLEU Score is 3.412594635300846
135m 37s (- 406m 53s) (5 25%) 1.1018
136m 18s (- 408m 55s) (5 25%) 1.0830
136m 59s (- 410m 58s) (5 25%) 1.0979
137m 40s (- 413m 0s) (5 25%) 1.1199
138m 20s (- 415m 2s) (5 25%) 1.1004


BLEU Score is 3.579020951482353
139m 10s (- 417m 31s) (5 25%) 1.1065
139m 51s (- 419m 33s) (5 25%) 1.0846
140m 31s (- 421m 35s) (5 25%) 1.0942
141m 12s (- 423m 37s) (5 25%) 1.0891
141m 53s (- 425m 39s) (5 25%) 1.0986


BLEU Score is 3.5156731943607618
142m 42s (- 428m 8s) (5 25%) 1.1189
143m 23s (- 430m 10s) (5 25%) 1.1085
144m 4s (- 432m 12s) (5 25%) 1.1123
144m 44s (- 434m 14s) (5 25%) 1.1206
145m 25s (- 436m 16s) (5 25%) 1.1152


BLEU Score is 3.3063521191518124
146m 15s (- 438m 45s) (5 25%) 1.1016
146m 55s (- 440m 47s) (5 25%) 1.1105
147m 36s (- 442m 49s) (5 25%) 1.1107
148m 17s (- 444m 51s) (5 25%) 1.1150
148m 57s (- 446m 53s) (5 25%) 1.1139


BLEU Score is 3.808447510385108
149m 48s (- 449m 24s) (5 25%) 1.1208
150m 28s (- 451m 26s) (5 25%) 1.1245


In [ ]:
hidden_size = 350
encoder1 = EncoderRNN(EMBEDDING_SIZE,hidden_size).to(device)
decoder1 = DecoderRNN(EMBEDDING_SIZE,hidden_size, len(ordered_words_ft)).to(device)

##UNCOMMENT TO TRAIN THE MODEL
trainIters(encoder1, decoder1, 20,'hidden_350', lr_decrease = True,print_every=50,plot_every = 100, evaluate_every = 250,learning_rate=0.005)

In [ ]:
# beam search + bleu score
def beam_search_decoder(data, k):
    sequences = [[list(), 1.0]]
    # walk over each step in sequence
    for row in data:
        all_candidates = list()
        # expand each current candidate
        for i in range(len(sequences)):
            seq, score = sequences[i]
            for j in range(len(row)):
                candidate = [seq + [j], score * -log(row[j])]
                all_candidates.append(candidate)
        # order all candidates by score
        ordered = sorted(all_candidates, key=lambda tup:tup[1])
        # select top best
        sequences = ordered[:1]
    return sequences

In [ ]:
class decoder_output_node:
    def __init__(self,parent, word_idx, prob_sum, isroot=False):
        self.parent = parent
        self.isroot = isroot
        self.children = []
        self.word_idx = word_idx
        self.prob_sum = prob_sum
    
    def get_children(self):
        '''
        return children
        '''
        return self.children
    
    def add_children(self, child):
        '''
        child: node
        '''
        self.children.append(child)
        return
    
    def get_parent(self):
        '''
        get parent of children
        '''
        return self.parent
    
    def get_word_idx(self):
        
        return self.word_idx
    
    def get_prob_sum(self):
        
        return self.prob_sum
    
    def is_root(self):
        return self.isroot


In [ ]:
def return_sentence_sequence(child_node):
    if child_node.is_root():
        return [child_node.get_word_idx()]
    
    return return_sentence_sequence(child_node.get_parent())+[child_node.get_word_idx()]

In [ ]:
def beam_search(beam_k, decoder_output, prob_sum = None, parent_node_list=None, vocab_size = len(idx2words_ft_en)):
    '''
    params:
    beam_k
    decoder_output: previous round decoder output
    parent_node_list: previous candidate word list (for only one candidate)
    
    return:
    list_of_best_k_nodes: best k nodes found in this iteration, list of list, first dim batch, second dim best k
    prob_with_sum: probabilistic matrix after sum+sortee 
    '''
    # if first word
    if parent_node_list is None:
        # initialize result
        prob_with_sum_sorted, word_idx_sorted = decoder_output.data.topk(beam_k)
        #print("ps",prob_with_sum_sorted)
        # add initialize tree list
        list_of_best_k_nodes = []
        batchsize = prob_with_sum_sorted.shape[0]
        for batch_i in range(batchsize):
            batch_i_tree_list = []
            for beam_i in range(beam_k):
                # add tree root node to list
                batch_i_tree_list.append(decoder_output_node(parent=None, word_idx=word_idx_sorted[batch_i, beam_i].item(), 
                                                            prob_sum= prob_with_sum_sorted[batch_i, beam_i].item(), isroot=True))
                
            list_of_best_k_nodes.append(batch_i_tree_list)
   
    # if not first word
    else:
        # get sorted results for all outputs
        prob = decoder_output.data
        #print(decoder_output.data.shape)
        #print(word_idx)
        
        
        # find top beam k words options
        #print("sum:",prob_sum)
        #print("curr prob:",prob)
        #print("sum:",prob+prob_sum)
        prob_with_sum = prob+prob_sum
        prob_with_sum_sorted, word_idx_sorted = torch.sort(prob_with_sum, dim=1, descending=True)
        #print("sum sorted:", prob_with_sum_sorted)
        # add top beam k words options into tree
        batchsize = prob_with_sum_sorted.shape[0]
        
        list_of_best_k_nodes = []
        for batch_i in range(batchsize):
            batch_i_tree_list = []
            for beam_i in range(beam_k):
                #print(word_idx_sorted[batch_i, beam_i])
                #print(parent_node_list[batch_i].get_word_idx())
                child_node = decoder_output_node(parent=parent_node_list[batch_i], word_idx= word_idx_sorted[batch_i,beam_i].item(), prob_sum=prob_with_sum_sorted[batch_i,beam_i].item())
                
                # update parent node's child
                parent_node_list[batch_i].add_children(child_node)
                #save child to new list
                batch_i_tree_list.append(child_node)
            # add batch tree list to best k
            list_of_best_k_nodes.append(batch_i_tree_list)
                
    return list_of_best_k_nodes, prob_with_sum_sorted[:,:beam_k], word_idx_sorted[:,:beam_k]


In [ ]:
def evaluate_with_beam_search(val_loader,encoder1,decoder1,beam_k = 5):
    big_pred_list = []
    big_ref_list = []
    #beam_k = 5
    with torch.no_grad():
        predictions = ''
        references = ''
        for i, (data_s1, data_s2, lengths_s1, lengths_s2) in enumerate(val_loader):
            #print(i)
            input_tensor = data_s1
            input_length = input_tensor.size()[0]
            #sentence_length to the output length
            sentence_length = data_s2.size()[1]
            encoder_hidden = encoder1.initHidden(input_length)

            encoder_output, encoder_hidden = encoder1(input_tensor, encoder_hidden)

            #decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
            decoder_input = torch.tensor(np.array([[SOS_IDX]]*input_length).reshape(1,input_length),device=device)

            decoder_hidden = encoder_hidden

            decoder_attentions = torch.zeros(sentence_length, sentence_length)
            decoded_words_eval = []
            list_of_best_k_nodes = []

            prob_with_sum_sorted = []
            #print("outside",prob_with_sum_sorted)

            decoder_hidden_list = []
            for di in range(sentence_length):

                ############################################beam search###################################################
                #print(di)
                if di == 0:
                    decoded_words_sub = []


                    decoder_output, decoder_hidden, decoder_attention = decoder1(
                                    decoder_input, decoder_hidden, encoder_output)

                    # find top k candidates
                    list_of_best_k_nodes,prob_with_sum_sorted ,word_idx_sorted = beam_search(beam_k, decoder_output, parent_node_list=None)
                    decoder_hidden_list = [decoder_hidden]*beam_k

                    #print("sum1",prob_with_sum_sorted)
                    #print("idx",word_idx_sorted)
                    #print(list_of_best_k_nodes[0][0].get_word_idx())
                    #print(list_of_best_k_nodes[0][1].get_word_idx())

                else:
                    # keep track of all new nodes
                    new_nodes = []
                    nodes_prob = None
                    #nodes_word_idx = None

                    # store index in previous candidate to locate position in new nodes, repeats=beam_size*beam_size
                    prev_candidate_idx = np.repeat(range(beam_k), repeats=beam_k)

                    # iterate through each node candidate from last iterations to find new candidates
                    new_decoder_hidden_list = []

                    for beam_i in range(beam_k):
                        #print(word_idx_sorted.shape)
                        topi = word_idx_sorted[:,beam_i].data
                        #print("idx i",topi)

                        prob_sum = prob_with_sum_sorted[:,beam_i].view((input_length,1))
                        #print("prob sum:", prob_sum)
                        #change the dimension
                        decoder_input = topi.squeeze().detach()
                        decoder_input = decoder_input.unsqueeze(0)

                        # get decoder output
                        decoder_output, decoder_hidden_i, decoder_attention = decoder1(
                                    decoder_input, decoder_hidden_list[beam_i], encoder_output)

                        new_decoder_hidden_list.append(decoder_hidden_i)

                        # get beam search output
                        best_k_curr_node, prob_sum_curr_node, _ = beam_search(beam_k, decoder_output, prob_sum=prob_sum, parent_node_list=[ls[beam_i] for ls in list_of_best_k_nodes])
                        #print(word_idx_curr_node)

                        # keep track of beam search output
                        new_nodes.append(best_k_curr_node)

                        if beam_i == 0:
                            nodes_prob = prob_sum_curr_node.data

                            #nodes_word_idx = word_idx_curr_node
                        else:
                            nodes_prob = torch.cat((nodes_prob, prob_sum_curr_node.data),dim=1)
                            #nodes_word_idx = torch.cat((nodes_word_idx, word_idx_curr_node),dim=1)

                    #print("nodes prob", nodes_prob)
                    _, sorted_idx = torch.sort(nodes_prob, dim=1, descending=True)
                    #print("length",nodes_prob.shape)
                    #print(nodes_prob)
                    #print(sorted_idx)

                    #print(prev_candidate_idx)
                    #print("new nodes len:", len(new_nodes[0][0]))
                    #print("new_nodes 0",new_nodes[0])
                    #print("new_nodes 1",new_nodes[1])
                    # update 
                    #print(sorted_idx.shape)
                    for batch_i in range(input_length):
                        for beam_i in range(beam_k):
                            # find the index of which candidate it descended from
                            st_idx = sorted_idx[batch_i][beam_i].item()
                            # find the corresponding node, st_idx gives parent node id, batch_i gives which example, st_idx%beam_k gives which node in the existing node list
                            #if batch_i == 0:
                            #print("st_idx",st_idx)
                            update_node = new_nodes[prev_candidate_idx[st_idx]][batch_i][st_idx%beam_k]

                            list_of_best_k_nodes[batch_i][beam_i] = update_node
                            #print(batch_i)
                            #print(beam_i)
                            #print(list_of_best_k_nodes[0][0].parent.get_word_idx())

                            # update word idex, prob sum correspondingly for next iteration
                            #word_idx_sorted[batch_i][beam_i] = nodes_word_idx[batch_i][st_idx] 
                            word_idx_sorted[batch_i][beam_i] = update_node.get_word_idx()
                            prob_with_sum_sorted[batch_i][beam_i] = update_node.get_prob_sum()


                            decoder_hidden_list[beam_i][0,batch_i,:] = new_decoder_hidden_list[prev_candidate_idx[st_idx]][0,batch_i,:]

                    #print("best k",list_of_best_k_nodes[0])
                    #print("final", prob_with_sum_sorted)
                    #print("idx final", word_idx_sorted)

            # find the best and get index
            listed_predictions = []
            for batch_i in range(input_length):
                best_sequence_last_node = list_of_best_k_nodes[batch_i][0]
                batch_i_word_idx = return_sentence_sequence(best_sequence_last_node)

                listed_predictions.append(' '.join(idx2words_ft_en[token_idx] for token_idx in batch_i_word_idx if token_idx!=PAD_IDX))
                #print(' '.join(idx2words_ft_en[token_idx] for token_idx in batch_i_word_idx ))
                #print(batch_i_word_idx)
                #print (listed_predictions)
            listed_reference = []
            for ele in data_s2:
                sent = index2token_sentence(ele)

                listed_reference.append(sent)
                #print ("\n")
                #print (sent)

            #print(listed_predictions)
            #bleu_score = corpus_bleu(listed_predictions,[listed_reference])
            #print('BLEU Score is %s' % (str(bleu_score.score)))

            big_pred_list += listed_predictions
            big_ref_list += listed_reference
            
    bleu_score = corpus_bleu(big_pred_list,[big_ref_list])
    print('BLEU Score is %s' % (str(bleu_score.score)))
            ############################################beam search###################################################
    return bleu_score